In [1]:
import json
import numpy as np
import gc

In [2]:
gc.disable()
gc.isenabled()

False

In [3]:
# Import Data

train_data,test_data = [],[]
with open('train_data.json',encoding='utf-8') as read_file:
    for line in read_file:
        train_data.append(json.loads(line))
    read_file.close()

with open('test_data.json',encoding='utf-8') as read_file:
    for line in read_file:
        test_data.append(json.loads(line))
    read_file.close()

In [4]:
train_review = train_data[0]['Reviews']
train_rating = np.asarray(train_data[0]['Ratings']).astype('float16')

test_review = test_data[0]['Reviews']
test_rating = np.asarray(test_data[0]['Ratings']).astype('float16')

In [5]:
train_rating.shape

(25000,)

In [6]:
for i in range(train_rating.shape[0]):
    if train_rating[i]>5:
        train_rating[i]=1
        test_rating[i]=1
    else:
        train_rating[i]=0
        test_rating[i]=0
        
np.max(train_rating)

1.0

### Clean Data

In [7]:
import Data_Preprocss as clean

In [8]:
train = clean.get_preocessed_data(train_review)
test = clean.get_preocessed_data(test_review)

In [9]:
print(len(train[0]))

499


### Vectorize data

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
cv = CountVectorizer(max_features=12000)

In [12]:
data = ((cv.fit_transform(train).toarray()).astype('float16'))

In [13]:
data.shape

(25000, 12000)

In [14]:
test = ((cv.transform(test).toarray()).astype('float16'))

In [15]:
test.shape

(25000, 12000)

### MLP model

In [16]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import *

In [17]:
model = Sequential()

In [18]:
model.add(Dense(64,activation='relu',input_shape=(12000,)))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [19]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                768064    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 770,689
Trainable params: 770,689
Non-trainable params: 0
_________________________________________________________________


In [21]:
hist=model.fit(data,train_rating,batch_size=512,epochs=3,validation_split=0.1)

Epoch 1/3
44/44 [==============================] - 4s 86ms/step - loss: 0.4375 - accuracy: 0.8167 - val_loss: 0.7178 - val_accuracy: 0.7008
Epoch 2/3
44/44 [==============================] - 3s 75ms/step - loss: 0.2567 - accuracy: 0.9072 - val_loss: 0.7396 - val_accuracy: 0.7280
Epoch 3/3
44/44 [==============================] - 3s 75ms/step - loss: 0.1936 - accuracy: 0.9297 - val_loss: 1.0783 - val_accuracy: 0.6528


In [23]:
model.evaluate(test,test_rating)

782/782 [==============================] - 3s 3ms/step - loss: 0.4665 - accuracy: 0.8318


[0.4664923846721649, 0.8317999839782715]

- `Save Model History`

In [24]:
import os

In [25]:
if not os.path.exists('./Model_history'):
    os.mkdir('./Model_history')
    print('Directory created')
else:
    print('Already Created')

Directory created


In [27]:
np.save('./Model_history/mlp_hist.npy',hist.history)
read_dictionary = np.load('./Model_history/mlp_hist.npy',allow_pickle='TRUE').item()
print(read_dictionary.keys())

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
